In [ ]:
def showrst(df):
    print(df.to_markdown(tablefmt="grid"))

In [ ]:
import pandas as pd
import nannyml as nml
from IPython.display import display

reference, analysis, analysis_gt = nml.datasets.load_synthetic_multiclass_classification_dataset()
display(reference.head(3))

In [ ]:
metadata = nml.extract_metadata(
    reference,
    model_name='credit_card_segment',
    model_type="classification_multiclass",
    exclude_columns=['identifier']
)
metadata.target_column_name = 'y_true'
display(metadata.is_complete())

In [ ]:
cbpe = nml.CBPE(
    model_metadata=metadata,
    chunk_size=6000,
    metrics=['roc_auc', 'f1']
)
cbpe = cbpe.fit(reference_data=reference)

In [ ]:
est_perf_analysis = cbpe.estimate(analysis)

In [ ]:
display(est_perf_analysis.data.head(3))

In [ ]:
showrst(est_perf_analysis.data.head(3))

In [ ]:
for metric in cbpe.metrics:
    est_perf_analysis.plot(kind='performance', metric=metric).show()

In [ ]:
est_perf_with_ref = cbpe.estimate(pd.concat([reference, analysis], ignore_index=True))

In [ ]:
for metric in cbpe.metrics:
    figure = est_perf_with_ref.plot(kind='performance', metric=metric)
    figure.show()

In [ ]:
engine='orca'
for metric in cbpe.metrics:
    fig = est_perf_analysis.plot(kind='performance', metric=metric)
    fig.write_image(file=f"../_static/tutorial-perf-est-mc-guide-analysis-{metric}.svg", engine=engine)

In [ ]:
for metric in cbpe.metrics:
    fig = est_perf_with_ref.plot(kind='performance', metric=metric)
    fig.write_image(file=f"../_static/tutorial-perf-est-mc-guide-with-ref-{metric}.svg", engine=engine)